Load the data

In [10]:
import pandas as pd
import pickle
import config as c

df_popular = pd.read_pickle(c.PATHS.OBJECTS_FOLDER / "transformed_data_popular.pickle")
with open(c.PATHS.OBJECTS_FOLDER / "embeddings_popular.pkl", "rb") as f:
    embeddings_popular = pickle.load(f)

df_unpopular = pd.read_pickle(c.PATHS.OBJECTS_FOLDER / "transformed_data_unpopular.pickle")
with open(c.PATHS.OBJECTS_FOLDER / "embeddings_unpopular.pickle", "rb") as f:
    embeddings_unpopular = pickle.load(f)


In [11]:
# drop all columns with "city_*" in the name
df_popular = df_popular.drop(columns=[col for col in df_popular.columns if "city_" in col])
df_unpopular = df_unpopular.drop(columns=[col for col in df_unpopular.columns if "city_" in col or "categor" in col])

In [12]:
df_popular.shape, embeddings_popular.shape, df_unpopular.shape, embeddings_unpopular.shape

((223152, 10), (223152, 768), (203012, 10), (203012, 768))

In [13]:
# store the embeddings (numpy array) in the dataframe
df_popular["embeddings"] = embeddings_popular.tolist()
df_unpopular["embeddings"] = embeddings_unpopular.tolist()
df_popular["is_popular"] = 1
df_unpopular["is_popular"] = 0

Create train and test sets

In [14]:
from sklearn.model_selection import train_test_split

df = pd.concat([df_popular, df_unpopular])
X = df.drop(columns=["is_popular"])
y = df["is_popular"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Build keras neural network model to predict binary popularity

In [15]:
X_train_embeddings = X_train["embeddings"].values.tolist()
X_test_embeddings = X_test["embeddings"].values.tolist()

# convert the list of embeddings to a numpy array
import numpy as np

X_train_embeddings = np.array(X_train_embeddings, dtype=np.float32)
X_test_embeddings = np.array(X_test_embeddings, dtype=np.float32)

X_train = X_train.drop(columns=["embeddings"])
X_test = X_test.drop(columns=["embeddings"])

In [16]:
# make sure all columns are numeric
X_train = X_train.apply(pd.to_numeric)
X_test = X_test.apply(pd.to_numeric)

# fill missing values with 0
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

In [17]:
from keras.layers import Input, Dense, Flatten, concatenate
from keras.models import Model

# Define input layers for text data and embeddings
input_text = Input(shape=(X_train.shape[1],))
input_embed = Input(shape=(len(X_train_embeddings[0]),))

# Define dense layers for text data and embeddings
dense_text = Dense(64, activation='relu')(input_text)
dense_embed = Dense(64, activation='relu')(input_embed)

# Concatenate the dense layers
concat = concatenate([dense_text, dense_embed])

# Define output layer
output = Dense(1, activation='sigmoid')(concat)

# Define the model
model = Model(inputs=[input_text, input_embed], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit([X_train, X_train_embeddings], y_train, epochs=10, batch_size=32, validation_data=([X_test, X_test_embeddings], y_test))


Epoch 1/10


2023-04-18 23:05:58.251530: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


10655/10655 [==============================] - 16s 664us/step - loss: nan - accuracy: 0.9031 - val_loss: nan - val_accuracy: 0.4741
Epoch 2/10
10655/10655 [==============================] - 7s 648us/step - loss: nan - accuracy: 0.4769 - val_loss: nan - val_accuracy: 0.4741
Epoch 3/10
10655/10655 [==============================] - 7s 624us/step - loss: nan - accuracy: 0.4769 - val_loss: nan - val_accuracy: 0.4741
Epoch 4/10
10655/10655 [==============================] - 7s 648us/step - loss: nan - accuracy: 0.4769 - val_loss: nan - val_accuracy: 0.4741
Epoch 5/10
10655/10655 [==============================] - 7s 661us/step - loss: nan - accuracy: 0.4769 - val_loss: nan - val_accuracy: 0.4741
Epoch 6/10
10655/10655 [==============================] - 7s 660us/step - loss: nan - accuracy: 0.4769 - val_loss: nan - val_accuracy: 0.4741
Epoch 7/10
10655/10655 [==============================] - 7s 641us/step - loss: nan - accuracy: 0.4769 - val_loss: nan - val_accuracy: 0.4741
Epoch 8/10
10655